In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [2]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230118_Plate537/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)

all_folders_drop, excel_drop = get_dropbox_video_folders("/DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate537/DATA/", True)
# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]
# all_folders_drop = pd.read_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20220118_Plate537_001,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,001
1,20220118_Plate537_002,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,002
2,20220118_Plate537_003,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,003
3,20220118_Plate537_004,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,004
4,20220118_Plate537_005,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,005
...,...,...,...,...,...
84,20220118_Plate537_086,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,086
85,20220118_Plate537_087,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,087
86,20220118_Plate537_088,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,088
87,20220118_Plate537_089,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,089


In [14]:
# directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_dataset/"
# all_folders_drop.to_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"].ge("452")]
folders_drop = folders_drop.loc[folders_drop["video"] == "084"]

In [15]:
folders_drop = folders_drop.sort_values(by=['folder'])

In [16]:
# FINAL CHECK

print(directory_targ)
folders_drop

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230118_Plate537/


,folder,Plate number,Date Imaged,tot_path_drop,video
82,20220118_Plate537_084,537,20230118,DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate5...,084


In [17]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2830035


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2830036


In [10]:
for xl_adress in excel_drop:
    file_name = xl_adress.split('/')[-1]
    
    download(xl_adress, directory_targ+file_name)